In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import collections
import pickle
import requests

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv("imp_ev_col.csv")
img_links = pd.read_csv("images.csv")

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def calculate_r2(model, X, y, r=None):
    X_train, x_test, y_train, y_test = train_test_split(np.array(X),np.array(y), test_size=0.3, random_state=r)

    model.fit(X_train, y_train)

    y_train_ = model.predict(X_train)
    y_train_r2 = r2_score(y_train_, y_train)
    y_train_error = mean_squared_error(y_train_, y_train)

    y_test_ = model.predict(x_test)
    y_test_r2 = r2_score(y_test_, y_test)
    y_test_error = mean_squared_error(y_test_, y_test)

    return {
        "train_r2" : y_train_r2,
        "test_r2" : y_test_r2,
        "train_mse" : y_train_error,
        "test_mse" : y_test_error
    }


In [ ]:
X = df[[
    'Top Speed',
    'Battery Capacity',
    'Gross Vehicle Weight (GVWR)'
    ]]

y = df['Electric Range']

In [ ]:
df['Gross Vehicle Weight (GVWR)'].mean()

In [ ]:
lr = LinearRegression()
calculate_r2(lr, X, y)

In [ ]:
dt = DecisionTreeRegressor(max_depth=3)
calculate_r2(dt, X, y)

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
calculate_r2(rf, X, y)

In [ ]:
pickle.dump(rf, open("rf_range_predict.pkl", "wb"))

In [ ]:
svr = SVR(kernel='linear')
calculate_r2(svr, X, y)

In [ ]:
def get_range(model, speed, battery, weight):
    result = model.predict([[speed, battery, weight]])
    return int(result[0])

In [ ]:
print("lr : ", get_range(lr, 185, 77.4, 2495))
print("svr : ", get_range(svr, 185, 77.4, 2495))
print("dt : ", get_range(dt, 185, 77.4, 2495))
print("rf : ",get_range(rf, 185, 77.4, 2495))

In [ ]:
def get_topSpeed(model, range, power, weight):
    result = model.predict([[range, power, weight]])
    return int(result[0])

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.head()

From Range get Top speed 

In [ ]:
X = df[['Electric Range','Battery Capacity','Gross Vehicle Weight (GVWR)']]
y = np.array(df['Top Speed']).reshape(-1,1)

In [ ]:
max = [-1,-1]
for i in range(0, 100):
    dt = RandomForestRegressor(n_estimators=500)
    res = calculate_r2(dt, X,y,i)

    if res['test_r2'] > max[0]:
        max[0] = res['test_r2']
        max[1] = i

max

In [ ]:
# dt = DecisionTreeRegressor(max_depth=3)
# calculate_r2(dt, X,y, r=49)

In [ ]:
rf1 = RandomForestRegressor(n_estimators=500)
calculate_r2(rf1, X, y, r=80)

In [ ]:
pickle.dump(rf1, open("rf_speed_predict.pkl", "wb"))

In [ ]:
get_range(dt, 185, 77.4, 2495)

In [ ]:
ids = df[df['Electric Range'] >= 417].sort_values(by='Electric Range').reset_index(drop=True)['id'][0:10]

In [ ]:
img_links[img_links['id'] == ids[0]]['link'].reset_index(drop=True)[0]

In [ ]:
url = img_links[img_links['id'] == ids[0]]['link'].reset_index(drop=True)[0]

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_image(url: str):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'lxml')

    imgs = soup.find_all("img")[1]
    return "https://ev-database.org" + imgs['src']

In [ ]:
# url = get_image(url)

In [ ]:
model = pickle.load(open("rf_range_predict.pkl", "rb"))

In [ ]:
t = 346
best_range_id = df[df['Electric Range'] <= t]['id']
filter = img_links[img_links['id'].isin(best_range_id)].reset_index(drop=True)

In [ ]:
for i in range(0, filter.shape[1]):
    id = filter['id'][i]
    brand = filter['Brand'][i]
    link = filter['link'][i]
    print(id, brand, link)
    break

In [336]:
df.head()

,id,Acceleration 0 - 100 km/h,Top Speed,Electric Range,Total Power,Total Torque,Battery Capacity,Gross Vehicle Weight (GVWR),Cargo Volume
0,1647,7.8,185,390,168,350,77.4,2495,432
1,1493,6.2,160,330,215,520,69.7,2580,495
2,1229,3.2,260,415,500,850,93.4,2880,366
3,1252,5.7,190,470,250,430,83.9,2605,470
4,1534,7.9,160,450,150,310,82.0,2300,385


In [360]:
df[['Acceleration 0 - 100 km/h', 'Top Speed', 'Electric Range', 'Battery Capacity','Gross Vehicle Weight (GVWR)']]

,Acceleration 0 - 100 km/h,Top Speed,Electric Range,Battery Capacity,Gross Vehicle Weight (GVWR)
0,7.8,185,390,77.4,2495
1,6.2,160,330,69.7,2580
2,3.2,260,415,93.4,2880
3,5.7,190,470,83.9,2605
4,7.9,160,450,82.0,2300
...,...,...,...,...,...
189,2.8,260,400,93.4,2870
190,7.9,160,450,82.0,2280
191,11.7,135,195,50.0,2440
192,6.2,180,400,82.0,2715


TESTING API

In [7]:
url = "http://127.0.0.1:8000/predict/range"
res = requests.post(url, json={"speed" : 300, "battery" : 82, "weight" : 2300})
ans = res.json()
ans

{'output': {'filter_list': '[{"id": "1252", "brand": "BMW", "link": "https://ev-database.org/car/1252/BMW-i4-eDrive40", "acceleration": "5.7", "speed": "190", "range": "470", "battery": "83.9", "weight": "2605"}, {"id": "1653", "brand": "Mercedes", "link": "https://ev-database.org/car/1653/Mercedes-EQE-AMG-53-4MATICplus", "acceleration": "3.5", "speed": "220", "range": "480", "battery": "100.0", "weight": "3095"}, {"id": "1484", "brand": "Mercedes", "link": "https://ev-database.org/car/1484/Mercedes-EQS-580-4MATIC", "acceleration": "4.3", "speed": "210", "range": "605", "battery": "120.0", "weight": "3060"}, {"id": "1537", "brand": "Mercedes", "link": "https://ev-database.org/car/1537/Mercedes-EQS-AMG-53-4MATICplus", "acceleration": "3.4", "speed": "250", "range": "565", "battery": "120.0", "weight": "3225"}, {"id": "1715", "brand": "Mercedes", "link": "https://ev-database.org/car/1715/Mercedes-EQE-350-4MATIC", "acceleration": "6.3", "speed": "210", "range": "500", "battery": "100.0", 

In [ ]:
speed = 163.204

In [308]:
ids = df[df['Top Speed'] >= speed]['id']

In [309]:
filter_list = img_links[img_links['id'].isin(ids)].reset_index(drop=True)

In [330]:
filter_list

,id,Brand,link
0,1647,Genesis,https://ev-database.org/car/1647/Genesis-GV60-...
1,1229,Porsche,https://ev-database.org/car/1229/Porsche-Tayca...
2,1252,BMW,https://ev-database.org/car/1252/BMW-i4-eDrive40
3,1438,Porsche,https://ev-database.org/car/1438/Porsche-Tayca...
4,1625,Porsche,https://ev-database.org/car/1625/Porsche-Tayca...
...,...,...,...
89,1519,BMW,https://ev-database.org/car/1519/BMW-i4-M50
90,1755,Ford,https://ev-database.org/car/1755/Ford-Mustang-...
91,1394,Porsche,https://ev-database.org/car/1394/Porsche-Tayca...
92,1116,Porsche,https://ev-database.org/car/1116/Porsche-Tayca...


In [304]:
filter_list = [
        {
            "id": filter_list['id'][i],
            "brand": filter_list['Brand'][i],
            "link": filter_list['link'][i]
        }
        for i in range(ids.__len__())
    ]

In [334]:
# filter_list

In [ ]:
for i in range(ids.__len__()):
    data = {"id" : img_links['id'][i], 
            "brand" : img_links['Brand'][i], 
            "link" : img_links['link'][i] }
    print(data)

In [268]:
def make_prediction(model, *args):
    input = np.array(args).reshape(1,-1)
    result = model.predict(input)[0]
    return result

In [269]:
make_prediction(model,450, 82, 2300)

164.04